In [1]:
import os
import tensorflow as tf
import pickle
import numpy as np
# import math as math
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
def vgg16_model(image_shape=(32, 32, 3), learning_rate=0.001):

    # model = tf.keras.Sequential()

    # # Remove the prediction layer and add to new model
    # for layer in vgg16_model.layers[:-1]: 
    #     model.add(layer)    

    # # Freeze the layers 
    # for layer in model.layers:
    #     layer.trainable = False

    # # Add 'softmax' instead of earlier 'prediction' layer.
    # model.add(tf.keras.layers.Dense(5, activation='softmax'))


    # reset_keras()

    # initialize the VGG16 model from the keras library
    vgg16_model = tf.keras.applications.VGG16(include_top=False,
                                              weights='imagenet',
                                              input_tensor=None,
                                              input_shape=image_shape,
                                              pooling=None,
                                              classes=5)

    model = tf.keras.Sequential()

    for layer in vgg16_model.layers[:-1]: 
        model.add(layer)


    # Freeze the layers
    for layer in model.layers:
        layer.trainable = False

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    # model.add(tf.keras.layers.Dense(layer4_size, activation=activation))
    # model.add(tf.keras.layers.Dropout(dropout))

    # Add 'softmax' instead of earlier 'prediction' layer.
    model.add(tf.keras.layers.Dense(43, activation='softmax'))

    # input_ = tf.keras.layers.Input(shape=IMAGE_SHAPE)
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                optimizer=tf.keras.optimizers.Adam(learning_rate), 
                metrics=['accuracy'])

    model.summary()

    return model

In [3]:
def generate_plots(plot_cntr, history, plot_acc, plot_loss, ep, bs, lr):
    """
    A method that takes the model history of a trained model and plots its:
    1. Training accuracy
    2. Training loss
    3. Validation accuracy
    4. Validation loss
    """
    acc = model_history.history['acc']
    val_acc = model_history.history['val_acc']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']


    plt.figure(plot_cntr)
    plt.suptitle('Accuracy learning curve', fontsize=20)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel('accuracy', fontsize=14)
    plt.plot(acc, label='training accuracy')
    plt.plot(val_acc, label='validation accuracy')
    plt.xticks(np.arange(0, epochs + epochs/10, epochs/10))
    plt.legend(loc="lower right")
    plt.savefig("{}accuracy_{}_{}_{}.png".format(plot_acc, ep, bs, lr), dpi=250)

    plt.figure(plot_cntr+500)
    plt.suptitle('Loss learning curve', fontsize=20)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel('loss', fontsize=14)
    plt.plot(loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xticks(np.arange(0, epochs + epochs/10, epochs/10))
    plt.legend(loc="upper right")
    plt.savefig("{}loss_{}_{}_{}.png".format(plot_loss, ep, bs, lr), dpi=250)


In [4]:
def best_results(model_history, file):
    """
    A method that prints the best validation accuracy and loss to a best_results.txt file.
    """
    acc = model_history.history['acc']
    val_acc = model_history.history['val_acc']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']

    
    f = open(file, "w+")
    f.write("Training accuracy = {}\n".format(acc))
    f.write("----------------------\n")
    f.write("Training loss = {}\n".format(loss))
    f.write("----------------------\n")
    f.write("Validation accuracy = {}\n".format(val_acc))
    f.write("----------------------\n")
    f.write("Validation loss = {}\n".format(val_loss))
    f.write("----------------------\n")
    f.write("----------------------\n\n")
    f.close()


In [5]:
# plot directory for accuracy and loss
results = "./vgg/"
plot_acc = "./vgg/acc/"
plot_loss = "./vgg/loss/"

os.makedirs(plot_acc, exist_ok=True)
os.makedirs(plot_loss, exist_ok=True)

In [6]:
# VGG16 only accepts RGB images, hence was trained on only 4 files
files = [0, 1, 2, 3]
acc_scores = [0 for i in range(4)]
epochs = [30]
batch_size = [128, 256, 512]
learning_rate = [0.001, 0.0001, 0.01]


In [7]:
for i in files:
    best_batch_size = 0
    best_learning_rate = 0

    # Opening file for reading in binary mode
    with open('./data/data{}.pickle'.format(i), 'rb') as f:
        data = pickle.load(f, encoding='latin1')  # dictionary type


    # Making channels come at the end
    X_train = data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
    X_val = data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
    X_test = data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

    y_train = data['y_train']
    y_val = data['y_validation']
    y_test = data['y_test']

    # # Showing loaded data from file
    # for i, j in data.items():
    #     if i == 'labels':
    #         print(i + ':', len(j))
    #     else:
    #         print(i + ':', j.shape)
    
    for ep in epochs:
        for bs in batch_size:
            for lr in learning_rate:

                print("-------------------")
                print("Compiling VGG16 model with {} epochs, {} batch size, learning rate {}".format(ep, bs, lr))
                print("-------------------")

                model = vgg16_model(learning_rate=lr)

                print("-------------------")
                print("Training VGG16 model with {} epochs, {} batch size, learning rate {}".format(ep, bs, lr))
                print("-------------------")

#                 with tf.device('/device:GPU:0'):
                history = model.fit(X_train, y_train, batch_size=bs, epochs=ep, validation_data=(X_val, y_val))

                predictions = model.predict(X_test, batch_size=bs)
                y_pred = np.argmax(predictions)
                score = accuracy_score(y_true, y_pred)
                
                if score > acc_scores[i]:
                    scores[i] = score
                    best_batch_size = bs
                    best_learning_rate = lr
                    best_history = history
                
                
    best_results(best_history, file="{}/data{}_results_{}_{}_{}.txt".format(results, file, best_batch_size, best_learning_rate))
    generate_plots(i, history, plot_acc, plot_loss, ep, bs, lr)
                



-------------------
Compiling VGG16 model with 1 epochs, 128 batch size, learning rate 0.01
-------------------
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

KeyboardInterrupt: 